# Building inverted index and answering queries

In this lab you are going to implement a standard document processing pipeline and then build a simple search engine based on it:
- starting from crawling documents, 
- then building an inverted index,
- and answering queries using this index.

## Preprocessing

First, we need a unified approach to documents and queries preprocessing. Implement a class responsible for that. Complete the code for given functions (most of them are just one-liners) and make sure you pass the tests. Make use of `nltk` library, `spacy`, or any other you know.

In [5]:
import nltk
nltk.download('punkt')

class Preprocessor:

    def __init__(self):
        self.stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by',
                           'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                           'of', 'on', 'that', 'the', 'to', 'was',
                           'were', 'will', 'with'}
        self.ps = nltk.stem.PorterStemmer()

    def tokenize(self, text):
        #TODO word tokenize text using nltk lib
        # YOUR CODE HERE
        return nltk.word_tokenize(text)

    def stem(self, word, stemmer):
        #TODO stem word using provided stemmer
        # YOUR CODE HERE
        return stemmer.stem(word)

    def is_apt_word(self, word):
        #TODO check if word is appropriate - not a stop word and isalpha, 
        # i.e consists of letters, not punctuation, numbers, dates
        # TODO YOUR CODE HERE
        return word not in self.stop_words and word.isalpha()

    def preprocess(self, text):
        tokenized = self.tokenize(text.lower())
        return [self.stem(w, self.ps) for w in tokenized if self.is_apt_word(w)]

[nltk_data] Downloading package punkt to /home/kamil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Tests

In [7]:
prep = Preprocessor()
text = 'To be, or not to be, that is the question'

assert prep.tokenize(text) == ['To', 'be', ',', 'or', 'not', 'to', 'be',
                               ',', 'that', 'is', 'the', 'question']
assert prep.stem('retrieval', prep.ps) == 'retriev'
assert prep.is_apt_word('qwerty123') is False
assert prep.preprocess(text) == ['or', 'not', 'question']

## Crawling and Indexing

### Base classes

Here are some base classes you will need for writing an indexer. The code is given, still, you need to change it slightly. Namely, the `parse()` method. Add the code which will extract relevant text from the documents, particularly trageting Reuters website structure.

In [26]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import os
import hashlib


class Document:

    def __init__(self, url):
        self.url = url

    def download(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                self.content = response.content
                return True
            else:
                return False
        except:
            return False

    def persist(self, path):
        # this code is not supposed to be good :) 
        # Please discuss why this line is bad
        url_hash = os.path.join(path, hashlib.sha256(quote(self.url).encode()).hexdigest())
        with open(url_hash, 'wb') as f:
            f.write(self.content)


class HtmlReutersArticle(Document):

    def normalize(self, href):
        if href is not None and href[:4] != 'http':
            href = urllib.parse.urljoin(self.url, href)
        return href

    def parse(self):

        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True

        model = BeautifulSoup(self.content, "html.parser")

        self.anchors = []
        a = model.find_all('a')
        for anchor in a:
            href = self.normalize(anchor.get('href'))
            text = anchor.text
            self.anchors.append((text, href))

        # extract only header and main content
        # discuss why using classes like article-body__content__17Yit 
        # is the wrong strategy
        header = model.find('h1') # TODO your code here
        content = model.find('p') # TODO your code here

        if header is None or content is None:
            self.article_text = ""
            return

        texts = header.findAll(string=True) + content.findAll(string=True)
        visible_texts = filter(tag_visible, texts)
        self.article_text = "\n".join(t.strip() for t in visible_texts)

In [27]:
doc = HtmlReutersArticle(
    "https://www.reuters.com/business/healthcare-pharmaceuticals/amazon-launches-virtual-healthcare-clinic-us-2022-11-15/")
doc.download()
doc.parse()
doc.persist("")
doc.article_text

'RPT Amazon launches virtual healthcare clinic in U.S. for common ailments\nThe Amazon logo is seen outside its JFK8 distribution center in Staten Island, New York, U.S. November 25, 2020.  REUTERS/Brendan McDermid\nAcquire Licensing Rights'

### Main class

The main indexer logic is here. We organize it as a crawler generator that adds certain visited pages to inverted index and saves them on disk. 

- `crawl_generator_for_index` method crawles the given website doing BFS, starting from `source` within given `depth`. Considers only inner pages (starting with https://www.reuters.com/...) for visiting. To speed up, do not consider pages with content type other than `html`: `.pdf`, `.mp3`, `.avi`, `.mp4`, `.txt`, ... . If crawler encounters an article page (of a form https://www.reuters.com/TOPIC/NAME...), it saves its content in a file in `collection_path` folder and populates the inverted index calling `index_doc` method. When done, saves on disk three resulting dictionaries:
    - `doc_urls`: `{doc_id : url}`
    - `index`: `{term : [collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]}`
    - `doc_lengths`: `{doc_id : doc_length}` 

    `limit` parameter is given for testing - if not `None`, break the loop when number of saved articles exceeds the `limit` and return without writing dictionaries to disk.
    
- `index_doc` method parses and preprocesses the content of a `doc` and adds it to the inverted index. Also keeps track of document lengths in a `doc_lengths` dictionary.


**TODO**: at line 38 we have a regular expression that matches for https://www.reuters.com/TOPIC/ARTICLE-NAME links, but fails for topic pages and external websites. Please write this regular expression.

In [34]:
from collections import Counter
from queue import Queue
import pickle
import os
import re


class ReutersSpecificIndexer:

    def __init__(self):
        # dictionaries to populate
        self.doc_urls = {}
        self.index = {}
        self.doc_lengths = {}
        # preprocessor
        self.prep = Preprocessor()

    def crawl_generator_for_index(self, source, depth, collection_path="collection", limit=None):
        #TODO generate url-s for visiting

        q = Queue()
        q.put((source, 0))
        visited = set()
        doc_counter = 0
        # creating a folder if needed
        if not os.path.exists(collection_path):
            os.makedirs(collection_path)
        # doing a BFS
        while not q.empty():
            url, url_depth = q.get()
            if url not in visited:
                visited.add(url)
                try:
                    doc = HtmlReutersArticle(url)  # download and parse url
                    if doc.download():
                        doc.parse()
                        # TODO PLEASE WRITE A CORRECT REGULAR EXPRESSION
                        if re.match(r'https://www.reuters.com/((\w+/?)*)', url):
                            doc.persist(collection_path)
                            self.doc_urls[doc_counter] = url
                            self.index_doc(doc, doc_counter)
                            doc_counter += 1
                            yield doc
                            if limit is not None and doc_counter == limit:
                                return

                            # filter links, consider only inner html pages
                        if url_depth + 1 < depth:
                            valid_anchors = filter(self.accepted_url, doc.anchors)
                            for a in valid_anchors:
                                q.put((a[1], url_depth + 1))

                except FileNotFoundError as e:
                    print("Analyzing", url, "led to FileNotFoundError")

    def accepted_url(self, anchor):
        url = str(anchor[1])
        if not url.startswith("https://www.reuters.com/"):
            return False
        if url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
            return False
        return True

    def index_doc(self, doc, doc_id):
        # add documents to index
        doc.parse()
        # preprocess - tokenize, remove stopwords and non-alphanumeric tokens and stem
        content = self.prep.preprocess(doc.article_text)
        self.doc_lengths[doc_id] = len(content)
        # get dict of terms in current article
        article_index = Counter(content)
        # update global index
        for term in article_index.keys():
            article_freq = article_index[term]
            if term not in self.index:
                self.index[term] = [article_freq, (doc_id, article_freq)]
            else:
                self.index[term][0] += article_freq
                self.index[term].append((doc_id, article_freq))

### Tests

Please make sure your crawler prints out urls with `print(k, c.url)`

In [35]:
indexer = ReutersSpecificIndexer()
k = 1
for c in indexer.crawl_generator_for_index(
        source="https://www.reuters.com/technology",
        depth=2,
        collection_path="test_collection",
        limit=15):
    print(k, c.url)
    k += 1

assert type(indexer.index) is dict
assert type(indexer.index['reuter']) is list
assert type(indexer.index['reuter'][0]) is int
assert type(indexer.index['reuter'][1]) is tuple

1 https://www.reuters.com/technology
2 https://www.reuters.com/technology#main-content
3 https://www.reuters.com/differentiator/
4 https://www.reuters.com/
5 https://www.reuters.com/world/
6 https://www.reuters.com/business/
7 https://www.reuters.com/markets/
8 https://www.reuters.com/sustainability/
9 https://www.reuters.com/legal/
10 https://www.reuters.com/breakingviews/
11 https://www.reuters.com/technology/
12 https://www.reuters.com/investigates/
13 https://www.reuters.com/account/register/sign-up/&journeyStart=navigation
14 https://www.reuters.com/technology/sec-collects-wall-streets-private-messages-whatsapp-probe-escalates-sources-2023-09-25/
15 https://www.reuters.com/tags/securities-enforcement/


Please test these documents contain a desired stem (or its derivate):

In [40]:
some_stem = 'bank'
print(indexer.index[some_stem])
for pair in indexer.index[some_stem][1:]:
    print(indexer.doc_urls[pair[0]])

[1, (5, 1)]
https://www.reuters.com/business/


### 1.2.4. Building an index

In [41]:
indexer = ReutersSpecificIndexer()
for k, c in enumerate(
        indexer
                .crawl_generator_for_index(
            "https://www.reuters.com/",
            3,
            "docs_collection",
            limit=100  # optional limit
        )):
    print(k + 1, c.url)

1 https://www.reuters.com/
2 https://www.reuters.com/#main-content
3 https://www.reuters.com/differentiator/
4 https://www.reuters.com/world/
5 https://www.reuters.com/business/
6 https://www.reuters.com/markets/
7 https://www.reuters.com/sustainability/
8 https://www.reuters.com/legal/
9 https://www.reuters.com/breakingviews/
10 https://www.reuters.com/technology/
11 https://www.reuters.com/investigates/
12 https://www.reuters.com/account/register/sign-up/&journeyStart=navigation
13 https://www.reuters.com/markets/quote/.SPX/
14 https://www.reuters.com/markets/quote/.IXIC/
15 https://www.reuters.com/markets/quote/.DJI/
16 https://www.reuters.com/markets/quote/.STOXX/
17 https://www.reuters.com/markets/quote/.FTSE/
18 https://www.reuters.com/markets/quote/.N225/
19 https://www.reuters.com/lseg/
20 https://www.reuters.com/technology/sec-collects-wall-streets-private-messages-whatsapp-probe-escalates-sources-2023-09-25/
21 https://www.reuters.com/markets/asia/china-bans-nomura-senior-inv

### Index statistics

Load the index and print the statistics. May you use this data to update stopwords list?

In [42]:
print('Total index keys count', len(indexer.index))

print('\nTop stems by number of documents they apperared in:')
sorted_by_n_docs = sorted(indexer.index.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
print([(sorted_by_n_docs[i][0], len(sorted_by_n_docs[i][1])) for i in range(20)])

print('\nTop stems by overall frequency:')
sorted_by_freq = sorted(indexer.index.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True)
print([(sorted_by_freq[i][0], sorted_by_freq[i][1][0]) for i in range(20)])

Total index keys count 1129

Top stems by number of documents they apperared in:
[('right', 51), ('licens', 51), ('acquir', 50), ('septemb', 27), ('photo', 27), ('reuter', 24), ('after', 21), ('trade', 13), ('peopl', 13), ('china', 13), ('via', 12), ('news', 11), ('lower', 11), ('world', 10), ('more', 10), ('monday', 10), ('financi', 10), ('up', 9), ('nation', 9), ('dure', 9)]

Top stems by overall frequency:
[('right', 54), ('licens', 50), ('acquir', 49), ('reuter', 31), ('photo', 28), ('septemb', 27), ('after', 21), ('china', 20), ('news', 16), ('new', 16), ('world', 13), ('trade', 13), ('peopl', 12), ('via', 11), ('financi', 11), ('south', 10), ('provid', 10), ('nation', 10), ('lower', 10), ('week', 9)]


## Answering a query (finally)

Now, given that we already have built the inverted index, it's time to utilize it for answering user queries. In this class there are two methods you need to implement:
- `boolean_retrieval`, the simplest form of document retrieval which returns a set of documents such that each one contains all query terms. Returns a set of document ids. Refer to *ch.1* of the book for details;
- `okapi_scoring`, Okapi BM25 ranking function - assigns scores to documents in the collection that are relevant to the user query. Returns a dictionary of scores, `doc_id:score`. Read about it in [Wikipedia](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) and implement accordingly.

Both methods accept `query` parameter in a form of a dictionary, `term:frequency`

In [43]:
from collections import Counter
import functools
import math


class QueryProcessing:

    @staticmethod
    def prepare_query(raw_query):
        # Initialize the preprocessor
        prep = Preprocessor()

        # The query should be treated similarly to the documents, so we preprocess it
        query = prep.preprocess(raw_query)

        # Once preprocessed, we count how many times each term appears in the query
        # This gives us a dictionary where keys are terms and values are their frequencies
        return Counter(query)

    @staticmethod
    def boolean_retrieval(query, index):
        # This function retrieves documents that contain ALL query terms

        # Start by creating an empty list to hold postings for each term in the query
        postings = []

        # For each term in the query
        for term in query.keys():
            # If the term isn't in the index, we skip it
            if term not in index.keys():
                continue
            # Get postings for the term from the index
            term_postings = index[term]
            # Extract only document IDs from the postings
            term_postings = [posting[0] for posting in term_postings[1:]]
            # Add the list of document IDs to our postings list
            postings.append(term_postings)
        # We want documents that contain ALL query terms, so we find the intersection of all postings
        return functools.reduce(lambda x, y: set(x).intersection(set(y)), postings)
        # return {0, 2, 3}

    @staticmethod
    def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):
        # This function computes a score for each document based on its relevance to the query

        # Initialize an empty dictionary to hold scores for each document
        scores = []

        # Total number of documents in our collection
        doc_num = len(doc_lengths)
        # Compute the average document length
        avgdl = sum(doc_lengths) / doc_num

        # For each term in the query
        for term in query.keys():
            # If the term isn't in the index, we skip it
            if term not in index.keys():
                continue

            # Compute inverse document frequency (IDF) for the term
            idf = math.log((doc_num - index[term][0] + 0.5) / (index[term][0] + 0.5) + 1)
            # Get postings for the term
            
            # For each posting, compute a score for the associated document
            for posting in postings:
                ...

                # If the document already has a score, add the new score to it. Otherwise, set the new score

        return scores


### Tests

In [56]:
test_doc_lengths = {1: 20, 2: 15, 3: 10, 4: 20, 5: 30}
test_index = {'x': [2, (1, 1), (2, 1)], 'y': [2, (1, 1), (3, 1)], 'z': [3, (2, 1), (4, 2)]}

test_query1 = QueryProcessing.prepare_query('x z')
test_query2 = QueryProcessing.prepare_query('x y')

assert QueryProcessing.boolean_retrieval(test_query1, test_index) == {2}
assert QueryProcessing.boolean_retrieval(test_query2, test_index) == {1}
okapi_res = QueryProcessing.okapi_scoring(test_query2, test_doc_lengths, test_index)
assert all(k in okapi_res for k in (1, 2, 3))
assert not any(k in okapi_res for k in (4, 5))
assert okapi_res[1] > okapi_res[3] > okapi_res[2]

### And now query the real index

In [60]:
import time

queries = ["russia ukraine", "Hong Kong", "crypto money"]
for q in queries:
    print(q)
    qobj = QueryProcessing.prepare_query(q)
    for res in QueryProcessing.boolean_retrieval(qobj, indexer.index):
        print('\t', indexer.doc_urls[res])

    s = time.time()
    okapi_res = QueryProcessing.okapi_scoring(qobj, indexer.doc_lengths, indexer.index)
    e = time.time()
    print(f"\t == Okapi Time: {e - s:.5f} ==")
    for res in okapi_res:
        print('\t', indexer.doc_urls[res], okapi_res[res])

    print()

russia ukraine
	 https://www.reuters.com/world/us/republicans-appeal-far-right-conservatives-avert-us-government-shutdown-2023-09-24/
	 == Okapi Time: 0.00002 ==
	 https://www.reuters.com/world/us/republicans-appeal-far-right-conservatives-avert-us-government-shutdown-2023-09-24/ 3.036059120543364
	 https://www.reuters.com/world/armenian-pm-says-likelihood-rising-that-ethnic-armenians-will-leave-karabakh-2023-09-24/ 2.025785179024357
	 https://www.reuters.com/world/ukraines-zelenskiy-says-he-met-top-businessmen-during-us-visit-2023-09-24/ 1.8732517929970098
	 https://www.reuters.com/world/defining-images-911-attacks-2023-09-11/ 1.6544604389625321
	 https://www.reuters.com/world/europe/russian-installed-head-donetsk-imposes-5-hour-curfew-2023-09-24/ 1.3769625675282013

Hong Kong


TypeError: unbound method set.intersection() needs an argument